In [1]:
import os
import numpy as np
import cv2
from shutil import copy2
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
# For process to not allocate entire GPU memory

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [3]:
TRAIN_DATA_PATH = "./trafficnet_dataset_v1/train/"
TEST_DATA_PATH = "./trafficnet_dataset_v1/test/"

In [4]:
image_files = []
labels = []
for class_ in os.listdir(TRAIN_DATA_PATH):
    class_folder_path = TRAIN_DATA_PATH+class_+"/"
    for image_file in os.listdir(class_folder_path):
        image_files.append(class_folder_path+image_file)
        labels.append(class_)


In [5]:
train_files, val_files, train_labels, val_labels = train_test_split(image_files, labels, stratify=labels, test_size=500)

In [6]:
len(train_files), len(val_files)

(3101, 500)

In [7]:
os.makedirs("./data_for_training/training/", exist_ok=True)
os.makedirs("./data_for_training/validation/", exist_ok=True)

for i, train_file in enumerate(train_files):
    os.makedirs("./data_for_training/training/"+train_labels[i]+"/", exist_ok=True)
    copy2(train_file, "./data_for_training/training/"+train_labels[i]+"/")

for i, val_file in enumerate(val_files):
    os.makedirs("./data_for_training/validation/"+val_labels[i]+"/", exist_ok=True)
    copy2(val_file, "./data_for_training/validation/"+val_labels[i]+"/")


In [14]:
# example of progressively loading images from file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest', dtype=np.float16)


In [15]:
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(TRAIN_DATA_PATH, class_mode='categorical', batch_size=128, target_size=(128, 128))
val_it = datagen.flow_from_directory(TEST_DATA_PATH, class_mode='categorical', batch_size=128, target_size=(128, 128))
test_it = datagen.flow_from_directory('trafficnet_dataset_v1/test/', class_mode='categorical', batch_size=128, target_size=(128, 128))
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 3600 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Batch shape=(128, 128, 128, 3), min=0.000, max=1.000


In [16]:
train_it.class_indices

{'accident': 0, 'dense_traffic': 1, 'fire': 2, 'sparse_traffic': 3}

In [74]:
tf.keras.backend.set_floatx('float16')

In [75]:
from tensorflow.keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights=None, include_top=False, input_shape=(128, 128, 3))

In [76]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-6]:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable, layer.dtype)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f1b01af1a50> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b01af1a90> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b01aef750> True float16
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f1b01ae4890> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b01ae2e90> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b021e4a50> True float16
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f1b0195ba90> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b0274ce10> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b021de890> True float16
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1b01bc3d10> True float16
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f1b021d20d0>

In [77]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu', kernel_initializer='he_normal', dtype=np.float16))
model.add(layers.Dropout(0.01, dtype=np.float16))
model.add(layers.Dense(512, activation='relu', kernel_initializer='he_normal', dtype=np.float16))
model.add(layers.Dense(64, activation='relu', kernel_initializer='he_normal', dtype=np.float16))
model.add(layers.Dense(4, activation='softmax', dtype=np.float16))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              8389632   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_18 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_19 (Dense)             (None, 4)                

In [78]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=['acc'])

In [79]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=0.00000001, verbose=1)
call_backs = [reduce_lr]

In [80]:
model.fit_generator(train_it, steps_per_epoch=train_it.samples/train_it.batch_size, validation_data=val_it, \
                    validation_steps=val_it.samples/val_it.batch_size, verbose=1, callbacks=call_backs, epochs=9000)

Epoch 1/9000
29/28 [==============================] - 20s 673ms/step - loss: 1.4232 - acc: 0.2491 - val_loss: 1.5567 - val_acc: 0.2500
Epoch 2/9000
29/28 [==============================] - 14s 494ms/step - loss: 1.3871 - acc: 0.2573 - val_loss: 1.5533 - val_acc: 0.2500
Epoch 3/9000
29/28 [==============================] - 15s 501ms/step - loss: 1.3882 - acc: 0.2512 - val_loss: 1.5537 - val_acc: 0.2500
Epoch 4/9000
29/28 [==============================] - 15s 511ms/step - loss: 1.3870 - acc: 0.2452 - val_loss: 1.5533 - val_acc: 0.2500
Epoch 5/9000
29/28 [==============================] - 15s 507ms/step - loss: 1.3870 - acc: 0.2458 - val_loss: 1.5533 - val_acc: 0.2500
Epoch 6/9000
29/28 [==============================] - 15s 506ms/step - loss: 1.3889 - acc: 0.2426 - val_loss: 1.5550 - val_acc: 0.2500
Epoch 7/9000
 9/28 [========>.....................] - ETA: 7s - loss: 1.3875 - acc: 0.2179

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/saadzia/anaconda3/envs/tf2_gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-80-fb90b15b3516>", line 2, in <module>
    validation_steps=val_it.samples/val_it.batch_size, verbose=1, callbacks=call_backs, epochs=9000)
  File "/home/saadzia/anaconda3/envs/tf2_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 1297, in fit_generator
    steps_name='steps_per_epoch')
  File "/home/saadzia/anaconda3/envs/tf2_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_generator.py", line 265, in model_iteration
    batch_outs = batch_function(*batch_data)
  File "/home/saadzia/anaconda3/envs/tf2_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 973, in train_on_batch
    class_weight=class_weight, reset_metrics=reset_metrics)
  File "/home/

KeyboardInterrupt: 

In [ ]:
model.save("trafficnet_vgg_16_086.h5")

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'trafficnet_vgg_16_086.h5'})
uploaded.SetContentFile('trafficnet_vgg_16_086.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

     |████████████████████████████████| 993kB 46.7MB/s eta 0:00:01


W0731 19:49:34.433693 139890822862720 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

Uploaded file with ID 1eEUp0e2mGY6sMuMRLRj3ZhZTKd3IlMvh
